<a href="https://colab.research.google.com/github/Thipwadee/Project2/blob/main/ML_Models_V_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from pandas.api.types import  CategoricalDtype

# machine learning model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression


In [2]:
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New')

In [80]:
df_encoded = pd.read_excel('/content/Cleaned_Thermal_Data.xlsx', sheet_name='sheet_2', usecols=lambda x: 'Unnamed' not in x,)
df_encoded

,Age,BMI,Temp,RH,V,Thermal_acceptance,Thermal_sensation_Vote,อุณหภูมิ,ความชื้น,กระแสลม,...,Clo,Met,พื้นที่ทำแบบสอบถาม_ภายนอก / ใต้ร่มไม้,พื้นที่ทำแบบสอบถาม_ภายนอก/ กลางแจ้ง,พื้นที่ทำแบบสอบถาม_ภายนอก/ มีหลังคาคลุม,พื้นที่ทำแบบสอบถาม_ภายใต้อาคาร,seasons_summer,seasons_winter,เพศ_ชาย,เพศ_หญิง
0,27,17.156262,30.5,42.9,0.13,Accept,0,2,1,2,...,0.55,1.2,0,0,1,0,0,1,1,0
1,36,23.183673,30.5,42.9,0.13,Accept,-1,3,0,5,...,0.55,1.2,0,0,1,0,0,1,1,0
2,25,38.019565,30.5,42.9,0.13,Accept,-1,2,0,4,...,0.55,1.2,0,0,1,0,0,1,0,1
3,29,27.548209,30.5,42.9,0.13,Accept,-1,3,3,4,...,0.55,1.2,0,0,1,0,0,1,1,0
4,28,33.744173,30.5,42.9,0.13,Accept,-1,2,1,4,...,0.55,1.2,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,37,27.160494,35.6,46.5,0.40,Accept,0,3,2,4,...,0.55,1.2,0,0,0,1,1,0,1,0
189,23,18.591037,35.6,46.5,0.40,Unaccept,0,3,1,4,...,0.55,1.2,0,0,0,1,1,0,0,1
190,27,16.456006,35.6,46.5,0.40,Accept,1,4,4,4,...,0.55,1.2,0,0,0,1,1,0,1,0
191,57,23.255019,35.6,46.5,0.40,Accept,1,3,1,5,...,0.55,1.2,0,0,0,1,1,0,1,0


In [81]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Age                                      193 non-null    int64  
 1   BMI                                      193 non-null    float64
 2   Temp                                     193 non-null    float64
 3   RH                                       193 non-null    float64
 4   V                                        193 non-null    float64
 5   Thermal_acceptance                       193 non-null    object 
 6   Thermal_sensation_Vote                   193 non-null    int64  
 7   อุณหภูมิ                                 193 non-null    int64  
 8   ความชื้น                                 193 non-null    int64  
 9   กระแสลม                                  193 non-null    int64  
 10  ความแรงของแสงแดด                         193 non-n

In [82]:
df_encoded['Thermal_sensation_Vote'] = df_encoded['Thermal_sensation_Vote'].astype(str)
print(df_encoded.dtypes)

Age                                          int64
BMI                                        float64
Temp                                       float64
RH                                         float64
V                                          float64
Thermal_acceptance                          object
Thermal_sensation_Vote                      object
อุณหภูมิ                                     int64
ความชื้น                                     int64
กระแสลม                                      int64
ความแรงของแสงแดด                             int64
เหงื่อบริเวณผิวหนัง                          int64
ความร้อน                                     int64
ความชื้น.1                                   int64
กระแสลม.1                                    int64
แสงแดด                                       int64
เหงื่อบริเวณผิวหนัง.1                        int64
TMRT                                       float64
Clo                                        float64
Met                            

#Spitt Data

In [83]:
X=df_encoded.drop(['Thermal_acceptance','Thermal_sensation_Vote'], axis =1)
y = df_encoded['Thermal_acceptance']
y2 = df_encoded['Thermal_sensation_Vote']



#Models
##แบ่งข้อมูลโดยใช้ 5-fold แล้วจัดการ imbalance class ด้วย smote

TA

In [108]:

 import numpy as np
 from sklearn.model_selection import KFold
 from imblearn.over_sampling import SMOTE
 from sklearn.preprocessing import StandardScaler

# # ตั้งค่าพารามิเตอร์สำหรับ K-Fold
 k = 5
 kf = KFold(n_splits=k, shuffle=True, random_state=42)

# # วนลูปผ่าน K-Fold
 for train_index, test_index in kf.split(X):
# #     # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     # ทำมาตรฐานข้อมูล
     scaler = StandardScaler()
     X_train_scaled = scaler.fit_transform(X_train)
     X_test_scaled = scaler.transform(X_test)

     # ใช้ SMOTE สำหรับการทำ resampling ข้อมูลชุดฝึก
     smote = SMOTE(random_state=42)
     X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

     print(f"Training set size after resampling: {X_train_res.shape}")
     print(f"Test set size: {X_test_scaled.shape}")


Training set size after resampling: (270, 26)
Test set size: (39, 26)
Training set size after resampling: (274, 26)
Test set size: (39, 26)
Training set size after resampling: (276, 26)
Test set size: (39, 26)
Training set size after resampling: (276, 26)
Test set size: (38, 26)
Training set size after resampling: (280, 26)
Test set size: (38, 26)


In [57]:
 param_grid = dict(
     C= [0.001, 0.01, 0.1, 1, 10, 100],
     penalty=['l1','l2'],
     max_iter= [100, 200, 500, 1000],
    solver= ['lbfgs', 'saga']
 )
 lgt_grid = GridSearchCV(LogisticRegression(), param_grid, cv=5, refit=True, verbose=0)
 lgt_grid.fit(X_train_res, y_train_res)
 print("best_params :",lgt_grid.best_params_)
 print("accuracy :",lgt_grid.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

best_params : {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
accuracy : 0.9285714285714286


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
120 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator

best_params : {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
accuracy : 0.9285714285714286

TSV

In [112]:

import numpy as np
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# ตั้งค่าพารามิเตอร์สำหรับ K-Fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# วนลูปผ่าน K-Fold
for train_index, test_index in kf.split(X):
    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y2_train, y2_test = y2.iloc[train_index], y2.iloc[test_index]

    # ทำมาตรฐานข้อมูล
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ใช้ SMOTE สำหรับการทำ resampling ข้อมูลชุดฝึก
    smote = SMOTE(k_neighbors=2, random_state=42)
    X_train_res, y2_train_res = smote.fit_resample(X_train_scaled, y2_train)

    print(f"Training set size after resampling: {X_train_res.shape}")
    print(f"Test set size: {X_test_scaled.shape}")



Training set size after resampling: (270, 26)
Test set size: (39, 26)
Training set size after resampling: (306, 26)
Test set size: (39, 26)
Training set size after resampling: (288, 26)
Test set size: (39, 26)
Training set size after resampling: (270, 26)
Test set size: (38, 26)
Training set size after resampling: (288, 26)
Test set size: (38, 26)


In [86]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['l2'],
    'logisticregression__max_iter': [100, 200, 500, 1000],
    'logisticregression__solver': ['lbfgs', 'saga']
}

pipe = Pipeline(steps=[('scale', StandardScaler()), ('logisticregression', LogisticRegression())])
grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, verbose=0)
grid.fit(X_train_res, y2_train_res)

print("Best parameters :", grid.best_params_)
print("Best accuracy :", grid.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Best parameters : {'logisticregression__C': 10, 'logisticregression__max_iter': 200, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.774107683000605


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best parameters : {'logisticregression__C': 10, 'logisticregression__max_iter': 200, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.774107683000605


In [60]:
#lr.get_params(deep=True)

In [87]:
param_grid = {'alpha': [0.0001,0.001,0.01,0.025,0.005,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], 'l1_ratio': [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
log_grid = GridSearchCV(ElasticNet(), param_grid, cv=5, refit=True, verbose=0)
log_grid.fit(X_train_res, y2_train_res)

print("best_params :",log_grid.best_params_)
print("best_score :",log_grid.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.160e+00, tolerance: 7.071e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.733e-01, tolerance: 5.123e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.321e+00, tolerance: 5.038e

best_params : {'alpha': 0.025, 'l1_ratio': 0.8}
best_score : 0.13081579073350563


best_params : {'alpha': 0.025, 'l1_ratio': 0.8}
best_score : 0.13081579073350563

# TSV-3

In [88]:
fo_mapping = {
    '-3' :'หนาว(-3)',
    '-2' :'เย็น(-2)',
    '-1' :'เย็น(-1)',
    '0' :'ปกติ(0)',
    '1' :'ร้อน(1)',
    '2' :'ร้อน(2)',
    '3' :'ร้อนมาก(3)',
}

df_encoded['TSV3'] = df_encoded['Thermal_sensation_Vote'].map(fo_mapping)

In [89]:
df_encoded['TSV3'][df_encoded['TSV3']=='หนาว(-3)'] = 'เย็น(-1)'
df_encoded['TSV3'][df_encoded['TSV3']=='เย็น(-2)'] = 'เย็น(-1)'

df_encoded['TSV3'][df_encoded['TSV3']=='ร้อน(2)'] = 'ร้อน(1)'
df_encoded['TSV3'][df_encoded['TSV3']=='ร้อนมาก(3)'] = 'ร้อน(1)'

<ipython-input-89-b69026129b4e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['TSV3'][df_encoded['TSV3']=='หนาว(-3)'] = 'เย็น(-1)'
<ipython-input-89-b69026129b4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['TSV3'][df_encoded['TSV3']=='เย็น(-2)'] = 'เย็น(-1)'
<ipython-input-89-b69026129b4e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['TSV3'][df_encoded['TSV3']=='ร้อน(2)'] = 'ร้อน(1)'
<ipyth

In [90]:
fo_mapping = {
    'เย็น(-1)' :'-1',
    'ปกติ(0)' : '0',
    'ร้อน(1)' :'1',
}

df_encoded['TSV3'] = df_encoded['TSV3'].map(fo_mapping)

In [91]:
df_encoded

,Age,BMI,Temp,RH,V,Thermal_acceptance,Thermal_sensation_Vote,อุณหภูมิ,ความชื้น,กระแสลม,...,Met,พื้นที่ทำแบบสอบถาม_ภายนอก / ใต้ร่มไม้,พื้นที่ทำแบบสอบถาม_ภายนอก/ กลางแจ้ง,พื้นที่ทำแบบสอบถาม_ภายนอก/ มีหลังคาคลุม,พื้นที่ทำแบบสอบถาม_ภายใต้อาคาร,seasons_summer,seasons_winter,เพศ_ชาย,เพศ_หญิง,TSV3
0,27,17.156262,30.5,42.9,0.13,Accept,0,2,1,2,...,1.2,0,0,1,0,0,1,1,0,0
1,36,23.183673,30.5,42.9,0.13,Accept,-1,3,0,5,...,1.2,0,0,1,0,0,1,1,0,-1
2,25,38.019565,30.5,42.9,0.13,Accept,-1,2,0,4,...,1.2,0,0,1,0,0,1,0,1,-1
3,29,27.548209,30.5,42.9,0.13,Accept,-1,3,3,4,...,1.2,0,0,1,0,0,1,1,0,-1
4,28,33.744173,30.5,42.9,0.13,Accept,-1,2,1,4,...,1.2,0,0,1,0,0,1,1,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,37,27.160494,35.6,46.5,0.40,Accept,0,3,2,4,...,1.2,0,0,0,1,1,0,1,0,0
189,23,18.591037,35.6,46.5,0.40,Unaccept,0,3,1,4,...,1.2,0,0,0,1,1,0,0,1,0
190,27,16.456006,35.6,46.5,0.40,Accept,1,4,4,4,...,1.2,0,0,0,1,1,0,1,0,1
191,57,23.255019,35.6,46.5,0.40,Accept,1,3,1,5,...,1.2,0,0,0,1,1,0,1,0,1


In [92]:
df_encoded.TSV3.value_counts()

1     91
0     59
-1    43
Name: TSV3, dtype: int64

Spitt Data

In [93]:
X=df_encoded.drop(['Thermal_acceptance','Thermal_sensation_Vote','TSV3'], axis =1)
y3 = df_encoded['TSV3']


In [114]:

import numpy as np
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# ตั้งค่าพารามิเตอร์สำหรับ K-Fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# วนลูปผ่าน K-Fold
for train_index, test_index in kf.split(X):
    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y3_train, y3_test = y3.iloc[train_index], y3.iloc[test_index]

    # ทำมาตรฐานข้อมูล
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ใช้ SMOTE สำหรับการทำ resampling ข้อมูลชุดฝึก
    smote = SMOTE(random_state=42)
    X_train_res, y3_train_res = smote.fit_resample(X_train_scaled, y3_train)

    print(f"Training set size after resampling: {X_train_res.shape}")
    print(f"Test set size: {X_test_scaled.shape}")



Training set size after resampling: (210, 26)
Test set size: (39, 26)
Training set size after resampling: (219, 26)
Test set size: (39, 26)
Training set size after resampling: (219, 26)
Test set size: (39, 26)
Training set size after resampling: (225, 26)
Test set size: (38, 26)
Training set size after resampling: (219, 26)
Test set size: (38, 26)


In [95]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['l2'],
    'logisticregression__max_iter': [100, 200, 500, 1000],
    'logisticregression__solver': ['lbfgs', 'saga']
}

pipe = Pipeline(steps=[('scale', StandardScaler()), ('logisticregression', LogisticRegression())])
grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, verbose=0)
grid.fit(X_train_res, y3_train_res)

print("Best parameters :", grid.best_params_)
print("Best accuracy :", grid.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Best parameters : {'logisticregression__C': 1, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.7859408033826638


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters : {'logisticregression__C': 1, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.7859408033826638

In [96]:
param_grid = {'alpha': [0.0001,0.001,0.01,0.025,0.005,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], 'l1_ratio': [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
log_grid = GridSearchCV(ElasticNet(), param_grid, cv=5, refit=True, verbose=0)
log_grid.fit(X_train_res, y3_train_res)

print("best_params :",log_grid.best_params_)
print("best_score :",log_grid.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.017e+00, tolerance: 1.234e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.246e-02, tolerance: 1.108e-02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.219e-01, tolerance: 1.069e

best_params : {'alpha': 0.1, 'l1_ratio': 0.2}
best_score : 0.22773241425404284



best_params : {'alpha': 0.1, 'l1_ratio': 0.2}

best_score : 0.22773241425404284

#TSV2

In [122]:
df_encoded['TSV2'] = df_encoded['TSV3']

In [123]:
df_encoded['TSV2'] = df_encoded['TSV3']
df_encoded['TSV2'][df_encoded['TSV2']== '-1' ] = '0'


<ipython-input-123-6bd053d3996d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_encoded['TSV2'][df_encoded['TSV2']== '-1' ] = '0'


In [124]:
df_encoded.TSV2.value_counts()

0    102
1     91
Name: TSV2, dtype: int64

Spitt Data

In [125]:
X = df_encoded.drop(['Thermal_acceptance','Thermal_sensation_Vote','TSV3','TSV2'], axis =1)
y4 = df_encoded['TSV2']

In [126]:

import numpy as np
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# ตั้งค่าพารามิเตอร์สำหรับ K-Fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# วนลูปผ่าน K-Fold
for train_index, test_index in kf.split(X):
    # แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y4_train, y4_test = y4.iloc[train_index], y4.iloc[test_index]

    # ทำมาตรฐานข้อมูล
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # ใช้ SMOTE สำหรับการทำ resampling ข้อมูลชุดฝึก
    smote = SMOTE(random_state=42)
    X_train_res, y4_train_res = smote.fit_resample(X_train_scaled, y4_train)

    print(f"Training set size after resampling: {X_train_res.shape}")
    print(f"Test set size: {X_test_scaled.shape}")



Training set size after resampling: (168, 26)
Test set size: (39, 26)
Training set size after resampling: (162, 26)
Test set size: (39, 26)
Training set size after resampling: (162, 26)
Test set size: (39, 26)
Training set size after resampling: (160, 26)
Test set size: (38, 26)
Training set size after resampling: (164, 26)
Test set size: (38, 26)


In [101]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['l2'],
    'logisticregression__max_iter': [100, 200, 500, 1000],
    'logisticregression__solver': ['lbfgs', 'saga']
}

pipe = Pipeline(steps=[('scale', StandardScaler()), ('logisticregression', LogisticRegression())])
grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, verbose=0)
grid.fit(X_train_res, y4_train_res)

print("Best parameters :", grid.best_params_)
print("Best accuracy :", grid.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Best parameters : {'logisticregression__C': 0.001, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.7422348484848486


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters : {'logisticregression__C': 0.001, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
Best accuracy : 0.7422348484848486

In [102]:
param_grid = {'alpha': [0.0001,0.001,0.01,0.025,0.005,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], 'l1_ratio': [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
log_grid = GridSearchCV(ElasticNet(), param_grid, cv=5, refit=True, verbose=0)
log_grid.fit(X_train_res, y4_train_res)

print("best_params :",log_grid.best_params_)
print("best_score :",log_grid.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.380e-03, tolerance: 3.252e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.552e-03, tolerance: 3.252e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.063e-02, tolerance: 3.252e

best_params : {'alpha': 0.1, 'l1_ratio': 0.2}
best_score : 0.24952308317230312



best_params : {'alpha': 0.1, 'l1_ratio': 0.2}
best_score : 0.24952308317230312


#Apply model

TA

In [110]:
Log_reg = LogisticRegression(C= 10, max_iter=100, penalty= 'l2', solver= 'lbfgs')

# Train the model
Log_reg.fit(X_train_res, y_train_res)

# Make predictions on the test set
y_pred = Log_reg.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[18 14]
 [ 0  6]]
              precision    recall  f1-score   support

      Accept       1.00      0.56      0.72        32
    Unaccept       0.30      1.00      0.46         6

    accuracy                           0.63        38
   macro avg       0.65      0.78      0.59        38
weighted avg       0.89      0.63      0.68        38



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


TSV

In [113]:
Log_reg = LogisticRegression(C= 10, max_iter=200, penalty= 'l2', solver= 'lbfgs')

# Train the model
Log_reg.fit(X_train_res, y2_train_res)

# Make predictions on the test set
y2_pred = Log_reg.predict(X_test)
print(confusion_matrix(y2_test,y2_pred))
print(classification_report(y2_test,y2_pred))

[[ 9  0  0  0  0]
 [11  0  0  0  0]
 [ 9  0  0  0  0]
 [ 5  0  0  0  0]
 [ 4  0  0  0  0]]
              precision    recall  f1-score   support

          -1       0.24      1.00      0.38         9
           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4

    accuracy                           0.24        38
   macro avg       0.05      0.20      0.08        38
weighted avg       0.06      0.24      0.09        38



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/loca

In [105]:
import numpy as np
model = ElasticNet(alpha=0.025, l1_ratio= 0.8)

# Train the model
model.fit(X_train_res, y2_train_res)

# Make predictions on the test set
y2_pred = model.predict(X_test)

bins = [ -2.5, -1.5, -0.5, 0.5, 1.5, 2.5]
y2_pred_categorical = np.digitize(y2_pred, bins)

y2_test = y2_test.astype(int)
y2_pred_categorical = y2_pred_categorical.astype(int)


print(confusion_matrix(y2_test, y2_pred_categorical))
print(classification_report(y2_test, y2_pred_categorical))



[[ 0  0  0  0  0  9]
 [ 0  0  0  0  0 11]
 [ 0  0  0  0  0  9]
 [ 0  0  0  0  0  5]
 [ 0  0  0  0  0  4]
 [ 0  0  0  0  0  0]]
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       9.0
           0       0.00      0.00      0.00      11.0
           1       0.00      0.00      0.00       9.0
           2       0.00      0.00      0.00       5.0
           3       0.00      0.00      0.00       4.0
           6       0.00      0.00      0.00       0.0

    accuracy                           0.00      38.0
   macro avg       0.00      0.00      0.00      38.0
weighted avg       0.00      0.00      0.00      38.0



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

3-TSV


Best parameters : {'logisticregression__C': 1, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
best_params : {'alpha': 0.1, 'l1_ratio': 0.2}

In [115]:
Log_reg = LogisticRegression(C= 1, max_iter=100, penalty= 'l2', solver= 'lbfgs')

# Train the model
Log_reg.fit(X_train_res, y3_train_res)

# Make predictions on the test set
y3_pred = Log_reg.predict(X_test)
print(confusion_matrix(y3_test,y3_pred))
print(classification_report(y3_test,y3_pred))

[[ 9  0  0]
 [11  0  0]
 [18  0  0]]
              precision    recall  f1-score   support

          -1       0.24      1.00      0.38         9
           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00        18

    accuracy                           0.24        38
   macro avg       0.08      0.33      0.13        38
weighted avg       0.06      0.24      0.09        38



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

In [118]:
import numpy as np
model = ElasticNet(alpha=0.1, l1_ratio= 0.2)

# Train the model
model.fit(X_train_res, y3_train_res)

# Make predictions on the test set
y3_pred = model.predict(X_test)

bins = [-0.5, 0.5, 1.5]
y3_pred_categorical = np.digitize(y3_pred, bins)

y3_test = y3_test.astype(int)
y3_pred_categorical = y3_pred_categorical.astype(int)


print(confusion_matrix(y3_test, y3_pred_categorical))
print(classification_report(y3_test, y3_pred_categorical))



[[ 0  0  1  1  7]
 [ 0  0  1  1  9]
 [ 0  0  0  0 18]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       9.0
           0       0.00      0.00      0.00      11.0
           1       0.00      0.00      0.00      18.0
           2       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0

    accuracy                           0.00      38.0
   macro avg       0.00      0.00      0.00      38.0
weighted avg       0.00      0.00      0.00      38.0



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

2-TSV

Best parameters : {'logisticregression__C': 0.001, 'logisticregression__max_iter': 100, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'lbfgs'}
best_params : {'alpha': 0.1, 'l1_ratio': 0.2}

In [120]:
Log_reg = LogisticRegression(C= 0.001, max_iter=100, penalty= 'l2', solver= 'lbfgs')

# Train the model
Log_reg.fit(X_train_res, y4_train_res)

# Make predictions on the test set
y4_pred = Log_reg.predict(X_test)
print(confusion_matrix(y4_test,y4_pred))
print(classification_report(y4_test,y4_pred))

[[ 0 20]
 [ 0 18]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.47      1.00      0.64        18

    accuracy                           0.47        38
   macro avg       0.24      0.50      0.32        38
weighted avg       0.22      0.47      0.30        38



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

In [128]:
import numpy as np
model = ElasticNet(alpha=0.1, l1_ratio= 0.2)

# Train the model
model.fit(X_train_res, y4_train_res)

# Make predictions on the test set
y4_pred = model.predict(X_test)

bins = [0,0.5,1]
y4_pred_categorical = np.digitize(y4_pred, bins)

y4_test = y4_test.astype(int)
y4_pred_categorical = y4_pred_categorical.astype(int)


print(confusion_matrix(y4_test, y4_pred_categorical))
print(classification_report(y4_test, y4_pred_categorical))



[[ 0  0 20]
 [ 0  0 18]
 [ 0  0  0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      20.0
           1       0.00      0.00      0.00      18.0
           3       0.00      0.00      0.00       0.0

    accuracy                           0.00      38.0
   macro avg       0.00      0.00      0.00      38.0
weighted avg       0.00      0.00      0.00      38.0



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par